In [5]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
from bs4 import BeautifulSoup
import requests
from datetime import date
current_date = date.today()
from PIL import Image
import pytesseract
import requests
from io import BytesIO
import sys
import time
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
sys.path.append('../scripts')  
from text_embedding import *
from translation import *
from df_handling import *
import re


In [6]:
def scroll_to_end(driver):
    # Get current scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the page to load
        time.sleep(2)  # Adjust sleep time as needed

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Reached the end of the page
        last_height = new_height

In [7]:
domains = ['pseudes', 'paraplanitiko', 'ellipis-pliroforia', 'pseudoepistimi', 'theories-synomosias', 'πολιτική', 'apati',  'parapliroforisi']

url_list = list()
for domain in domains:

    URL = f"https://factcheckcyprus.org/category/fact-checks/{domain}/"
    

    driver = webdriver.Chrome()
    driver.get(URL)
    scroll_to_end(driver)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    urls = soup.find_all('h2', class_='blog-entry-title entry-title')

    for url in urls:
        print (url.find('a').get('href'))
        url_list.append(url.find('a').get('href'))
    driver.quit()

https://factcheckcyprus.org/fact-checks/vaccines-not-responsible-for-rise-in-cancer-cases/
https://factcheckcyprus.org/fact-checks/zelensky_did_not_buy_goebbels_house/
https://factcheckcyprus.org/fact-checks/pseudes/no-moderna-has-not-admitted-that-mrna-vaccines-cause-turbo-cancer/
https://factcheckcyprus.org/fact-checks/parapliroforisi/video-not-from-hamas-armory-under-hospital/
https://factcheckcyprus.org/fact-checks/pseudes/the-story-about-the-suicide-of-israeli-prime-minister-netanyahus-psychiatrist-is-satire/
https://factcheckcyprus.org/fact-checks/parapliroforisi/no-evidence-that-nephew-of-benjamin-netanyahu-was-killed-in-gaza/
https://factcheckcyprus.org/fact-checks/no-the-crowd-wasnt-chanting-fuck-you-at-joe-biden/
https://factcheckcyprus.org/fact-checks/this-picture-of-a-sign-about-refugees-has-been-digitally-altered/
https://factcheckcyprus.org/fact-checks/paraplanitiko/internationalists-do-not-fund-hamas/
https://factcheckcyprus.org/fact-checks/paraplanitiko/this-video-does-

In [8]:
print(len(url_list))
df = pd.DataFrame(columns=['id','claim','conclusion','article_title','author','publish_date','harvest_date','url','label', 'article_text'])

152


In [9]:
word_to_replace = "Ισχυρισμός:"
word_to_replace2 = "Συμπέρασμα:"
#Harvest method for greekhoaxes.gr
def harvest(urls, df):
    for url in urls:
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        
        title = soup.find('h1', class_='single-post-title').text
        main_content = soup.find('div', class_='entry-content clr')
        date = soup.find('li', class_='meta-date').text
        author = soup.find('div', class_='post-author-name').text
        claim_text=''
        conclusion = ''
        main_text = ''
        label=0

        if main_content:
            text = main_content.find_all('p')
        if not main_content:
            main_content = soup.find('div', class_='site-content clr')
            if not main_content:
                continue
        
        
        for txt in text:
            txt = txt.text

            main_text += txt.strip()
                # # print(txt.text)
                # txt = txt.text
                # if(str(txt).upper().startswith('ΙΣΧΥΡΙΣΜ') and not claim_text):
                #     claim_text = txt.strip()
                #     claim_text = claim_text.replace('Ισχυρισμός:', '').replace('ΙΣΧΥΡΙΣΜΟΣ:', '').strip()
                    
            # if(str(txt).upper().startswith('ΣΥΜΠ') and str(txt.split()[0]).endswith(':') and not conclusion):
            #     conclusion = txt.strip()
            #     conclusion = conclusion.replace('Συμπέρασμα:', '').replace('ΣΥΜΠΕΡΑΣΜΑ:', '').strip()        
                    

        claims = soup.find_all(class_='has-vivid-red-background-color has-background')
        if claims:
            for claim in claims:
                claim_text += claim.text

        if not claim_text:
            claims = soup.find_all(class_='class="has-vivid-cyan-blue-background-color has-background"')
            if claims:
                    for claim in claims:
                        claim_text += claim.text
        if not claim_text:
            claims = soup.find_all(class_="has-luminous-vivid-orange-background-color has-background")
            if claims:
                for claim in claims:
                    claim_text += claim.text

        if not claim_text:
            if(len(text[0].text.split())>=10 ):
                if((str(text[0].text.startswith('Το άρθρο')))):
                    claim_text = text[0].text
              
                        
        if not claim_text: 
            claim_text = None


        conclusion = soup.find(class_='has-vivid-green-cyan-background-color has-background')
        if conclusion:
            conclusion = conclusion.text
        if not conclusion:
            conclusion = soup.find(class_='has-vivid-cyan-blue-background-color has-background')
            if conclusion:
                conclusion = conclusion.text
        if len(str(conclusion).split())<=2:
            conclusion = soup.find_next(class_='has-vivid-cyan-blue-background-color has-background')
            if conclusion:
                conclusion = conclusion.text


        if not conclusion:
            conclusion = ''

        if main_text == "":
            print(url)

        

        if (str(claim_text).startswith('ΙΣΧΥΡΙΣΜΟΣ:') |str(claim_text).startswith('Ισχυρισμός') | str(claim_text).startswith('ΙΣΧΥΡΙΣΜΌΣ:')):
            
            label = 0
        else:
            label = 1
            


        
       




            
        


#         print(f'''Title: {title.strip()}
# date: {date.replace('Published','').replace(',','').strip()}
# author: {author.replace('Written by', '').strip()}
# claim: {claim_text}
# conclusion:{conclusion}
# main text: {main_text}''')

        if(claim_text):
            data = {'id': len(df), 'claim': claim_text.replace('Ισχυρισμός:', '').replace('ΙΣΧΥΡΙΣΜΟΣ:', ''), 
            'conclusion': conclusion.replace('Συμπέρασμα:', '').replace('ΣΥΜΠΕΡΑΣΜΑ:', ''), 'article_title': title.strip(), 'author': author.replace('Written by', '').strip(), 
            'publish_date': date.replace('Published','').replace(',','').strip(), 
                    'harvest_date':current_date , 'url': url, 'article_text': main_text, 'label': label }
            print("Claim: ", claim_text)
            print("Conclusion: ", conclusion)
            print('Label: ', label)
        
            df.loc[len(df)] = data

In [10]:
harvest(url_list,df)

Claim:  Ισχυρισμός: Οι εμβολιασμοί για COVID19 Οδήγησαν σε αύξηση καρκίνων σύμφωνα με τις αναφορές American Cancer Society Cancer Facts.
Conclusion:  Συμπέρασμα: οι γραφικές παραστάσεις που δείχνουν αύξηση των περιπτώσεων καρκίνου είναι στην πραγματικότητα μοντέλα με προβλέψεις για τον καρκίνο που βασίζονται σε ιστορικά δεδομένα πριν από την πανδημία και δεν αντικατοπτρίζουν τον αντίκτυπο της πανδημίας ή των εμβολίων. Οι τάσεις του καρκίνου επηρεάζονται από διάφορους παράγοντες, συμπεριλαμβανομένων του τρόπου ζωής, των δημογραφικών αλλαγών και της κοινωνικοοικονομικής κατάστασης, και όχι τους εμβολιασμούς για την COVID-19. Ο πραγματικός αντίκτυπος της πανδημίας φαίνεται στην καθυστέρηση ή διακοπή των διαδικασιών πρόληψης, διάγνωσης και θεραπείας του καρκίνου, παρά στην πρόκληση νέων περιπτώσεων καρκίνου. Συνεπώς, ο ισχυρισμός που συνδέει τους εμβολιασμούς κατά της COVID-19 με αύξηση των περιστατικών καρκίνου δεν υποστηρίζεται.
Label:  0
Claim:  ΙΣΧΥΡΙΣΜΟΣ: Ο Zelensky αγόρασε ένα σπίτι 

In [11]:
df.shape

(129, 10)

In [12]:
df.label.value_counts()

0    84
1    45
Name: label, dtype: int64

In [13]:
df = save_df(df, 'cyprus_factcheck.csv', None )

In [14]:
df2= pd.read_csv('../data/cyprus_factcheck.csv')

In [15]:
emb = df2[['id', 'claim', 'label']]
emb.insert(2, 'text_embedding', '')

In [16]:
emb

,id,claim,text_embedding,label
0,0,Οι εμβολιασμοί για COVID19 Οδήγησαν σε αύξηση...,,0
1,1,Ο Zelensky αγόρασε ένα σπίτι που προηγουμένως...,,0
2,2,Τα εμβόλια προκαλούν turbo-καρκίνο. Οι ερευνη...,,0
3,3,Αποθήκη φαρμάκων κάτω από το νοσοκομείο της Γ...,,0
4,4,Δημοσιεύματα αναφέρουν ότι ο ψυχίατρος του Ισ...,,0
...,...,...,...,...
124,124,Παραπλανητική ανάρτηση αναφέρεται σε απόσπασμα...,,1
125,125,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...,,1
126,126,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...,,1
127,127,"Ο Μητροπολίτης Μόρφου Νεόφυτος, γνωστός πλέον ...",,1


In [17]:
create_text_embeddings(emb, [1], ['text_embedding'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


In [18]:
emb

,id,claim,text_embedding,label
0,0,Οι εμβολιασμοί για COVID19 Οδήγησαν σε αύξηση...,"[0.011257008649408817, -0.0183500237762928, 0....",0
1,1,Ο Zelensky αγόρασε ένα σπίτι που προηγουμένως...,"[-0.007903427816927433, -0.01469595916569233, ...",0
2,2,Τα εμβόλια προκαλούν turbo-καρκίνο. Οι ερευνη...,"[-0.014585002325475216, -0.005677587818354368,...",0
3,3,Αποθήκη φαρμάκων κάτω από το νοσοκομείο της Γ...,"[0.02059408649802208, 0.006566992029547691, 0....",0
4,4,Δημοσιεύματα αναφέρουν ότι ο ψυχίατρος του Ισ...,"[0.00023768078244756907, -0.00865598302334547,...",0
...,...,...,...,...
124,124,Παραπλανητική ανάρτηση αναφέρεται σε απόσπασμα...,"[-0.02292361482977867, 0.00639630202203989, 0....",1
125,125,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...,"[-0.01149488240480423, -0.0026551042683422565,...",1
126,126,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...,"[-0.014770058915019035, -0.005550467874854803,...",1
127,127,"Ο Μητροπολίτης Μόρφου Νεόφυτος, γνωστός πλέον ...","[-0.006744216661900282, 0.0005294009461067617,...",1


In [19]:
df2

,id,claim,conclusion,article_title,author,publish_date,harvest_date,url,label,article_text
0,0,Οι εμβολιασμοί για COVID19 Οδήγησαν σε αύξηση...,οι γραφικές παραστάσεις που δείχνουν αύξηση τ...,Τα εμβόλια δεν προκάλεσαν κατακόρυφη αύξηση τω...,Antreas Antreou,February 1 2024,2024-02-08,https://factcheckcyprus.org/fact-checks/vaccin...,0,Ισχυρισμός: Οι εμβολιασμοί για COVID19 Οδήγησα...
1,1,Ο Zelensky αγόρασε ένα σπίτι που προηγουμένως...,Ο ισχυρισμός διαδόθηκε από αναξιόπιστες πηγές...,O B. Ζελένσκι δεν αγόρασε τη βίλα του Τζ. Γκαί...,Iacovos Mylonas,January 17 2024,2024-02-08,https://factcheckcyprus.org/fact-checks/zelens...,0,ΙΣΧΥΡΙΣΜΟΣ: Ο Zelensky αγόρασε ένα σπίτι που π...
2,2,Τα εμβόλια προκαλούν turbo-καρκίνο. Οι ερευνη...,Ενώ στο παρελθόν είχαν εκφραστεί ανησυχίες σε...,"Όχι, η Moderna δεν παραδέχτηκε ότι τα εμβόλια ...",Iacovos Mylonas,November 26 2023,2024-02-08,https://factcheckcyprus.org/fact-checks/pseude...,0,Ισχυρισμός: Τα εμβόλια προκαλούν turbo-καρκίνο...
3,3,Αποθήκη φαρμάκων κάτω από το νοσοκομείο της Γ...,Το βίντεο με το οπλοστάσιο κυκλοφόρησε στις 3...,Αυτό το βίντεο δεν δείχνει οπλοστάσιο της Χαμά...,Iacovos Mylonas,November 18 2023,2024-02-08,https://factcheckcyprus.org/fact-checks/parapl...,0,Μεγάλη διάδοση στο X έχει ένα βίντεο που φέρει...
4,4,Δημοσιεύματα αναφέρουν ότι ο ψυχίατρος του Ισ...,Ο ισχυρισμός προέρχεται από μια ανάρτηση σε ι...,Η ιστορία περί αυτοκτονίας του ψυχίατρου του Ι...,Loukia Taxitari,November 11 2023,2024-02-08,https://factcheckcyprus.org/fact-checks/pseude...,0,Ισχυρισμός: Δημοσιεύματα αναφέρουν ότι ο ψυχία...
...,...,...,...,...,...,...,...,...,...,...
124,124,Παραπλανητική ανάρτηση αναφέρεται σε απόσπασμα...,NaN,"‘Οχι, ο διευθύνων σύμβουλος της Pfizer ΔΕΝ είπ...",Alexandros Efstratiou,August 25 2021,2024-02-08,https://factcheckcyprus.org/fact-checks/parapl...,1,Παραπλανητική ανάρτηση αναφέρεται σε απόσπασμα...
125,125,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...,NaN,Παραπληροφόρηση ακόμη μια φορά από της σελίδες...,Iacovos Mylonas,July 26 2021,2024-02-08,https://factcheckcyprus.org/fact-checks/parapl...,1,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...
126,126,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...,NaN,Παραπληροφόρηση ακόμη μια φορά από της σελίδες...,Iacovos Mylonas,July 21 2021,2024-02-08,https://factcheckcyprus.org/fact-checks/parapl...,1,Οι αναφερόμενοι ισχυρίζονται ότι η συγκεκριμέν...
127,127,"Ο Μητροπολίτης Μόρφου Νεόφυτος, γνωστός πλέον ...",NaN,Απαράδεκτες και ψευδείς δηλώσεις του Μητροπολί...,Iacovos Mylonas,April 1 2021,2024-02-08,https://factcheckcyprus.org/fact-checks/theori...,1,"Ο Μητροπολίτης Μόρφου Νεόφυτος, γνωστός πλέον ..."


In [29]:
save_df(emb,'cyprus_factcheck_emb.csv', ['text_embedding'])

c:\Users\johnk\Desktop\LLM\web scraping\../scripts\df_handling.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: ','.join(map(str, x)))


In [30]:
test2 = pd.read_csv('../data/cyprus_factcheck_emb.csv')

In [32]:
test2.label.value_counts()

0    84
1    45
Name: label, dtype: int64